# Model Training - Credit Card Fraud Detection

In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
import joblib

In [2]:
# Load processed data
X_train = pd.read_csv('../data/processed/X_train.csv')
y_train = pd.read_csv('../data/processed/y_train.csv').values.ravel()

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")

X_train shape: (1296675, 5)
y_train shape: (1296675,)


In [3]:
# Train a simple logistic regression model
model = LogisticRegression(max_iter=1000, random_state=42)

print("Training model...")
model.fit(X_train, y_train)
print("Model trained!")

Training model...
Model trained!


In [4]:
# Save the model
joblib.dump(model, '../models/fraud_model.pkl')
print("Model saved to ../models/fraud_model.pkl")

Model saved to ../models/fraud_model.pkl
